In [ ]:
# Task 1

In [ ]:
! gdown --id 1q2gnTZHNP5_DZFMc4VHO7Fah6X4vZcR0

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1q2gnTZHNP5_DZFMc4VHO7Fah6X4vZcR0
To: /content/SqueezeNet_v1.2-master.zip
100% 4.59M/4.59M [00:00<00:00, 57.5MB/s]


In [ ]:
! gdown --id 18_e1gBcQxf_S-FnhxIc3b-D8mVNFZ9WV

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=18_e1gBcQxf_S-FnhxIc3b-D8mVNFZ9WV
To: /content/kaggle.json
100% 65.0/65.0 [00:00<00:00, 240kB/s]


In [ ]:
pip install kaggle

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Set your Kaggle API credentials
os.environ["KAGGLE_USERNAME"] = "maryamsaa"
os.environ["KAGGLE_KEY"] = "b73d47e88e6150c7dbb3ae61c96b1024"

# Specify the competition name
competition_name = "dogs-vs-cats-redux-kernels-edition"

# Set the working directory to where you want to download the files
#os.chdir("my-work")

# Create a Kaggle API object
api = KaggleApi()
api.authenticate()



In [ ]:
# Download the train.zip file
api.competition_download_file(competition=competition_name, file_name="train.zip")

# Download the test.zip file
api.competition_download_file(competition=competition_name, file_name="test.zip")


100%|██████████| 544M/544M [00:07<00:00, 74.4MB/s]


100%|██████████| 271M/271M [00:04<00:00, 62.6MB/s]

In [ ]:
import zipfile
import os

# Specify the paths to the zip files
train_zip_path = "train.zip"
test_zip_path = "test.zip"
SqueezeNet_zip_path = "SqueezeNet_v1.2-master.zip"


# Extract the contents of train.zip in the same directory
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(train_zip_path))

# Extract the contents of test.zip in the same directory
with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(test_zip_path))

# Extract the contents of SqueezeNet_v1.2-master.zip in the same directory
with zipfile.ZipFile(SqueezeNet_zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.dirname(SqueezeNet_zip_path))


In [ ]:
pip install mxnet==1.5.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [ ]:
import mxnet as mx
import numpy as np
np.bool = np.bool_

symbol_file = 'SqueezeNet_v1.2-master/model'
params_file = 'SqueezeNet_v1.2-master/model-0000.params'

# Load the model
sym, arg_params, aux_params = mx.model.load_checkpoint(symbol_file, 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))],
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [ ]:
def get_image(url, show=False):
    if url.startswith('http'):
        # download and show the image
        fname = mx.test_utils.download(url)
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], labels[i]))


In [ ]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['fire9_concat_output',
 'dropout0_output',
 'conv10_conv_weight',
 'conv10_conv_bias',
 'conv10_conv_output',
 'conv10_relu_output',
 'pool10_output',
 'flatten0_output',
 'softmax_label',
 'softmax_output']

In [ ]:
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [ ]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [ ]:
img = get_image('https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png')
features = get_features(img)
#print("{}\n shape: {}".format(features,features.shape))

In [ ]:
# from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data
from os import listdir
from os.path import isfile, join
import os

mypath = join(os.getcwd(),'train')

cats_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('cat')]
dogs_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('dog')]

In [ ]:
print("cats: {} and dogs: {}".format(len(cats_imgs),len(dogs_imgs)))

cats: 12500 and dogs: 12500


In [ ]:
Nmax = 50
cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]

In [ ]:
Y_cats = np.array(Nmax * [1])
Y_dogs = np.array(Nmax * [0])

In [ ]:
X_cvd = np.vstack([cats_features,dogs_features])
Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression().fit(X_train, y_train)

print("Test set score: {:.2f}".format(lg.score(X_test, y_test)))

Test set score: 0.96


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import time
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def extract_features(img_paths, fe_mod):
    features = []
    for img_path in img_paths:
        img = get_image(img_path)
        img_features = get_features(img)
        features.append(img_features.ravel())
    return np.array(features)

def train_classifier(X_train, y_train, param_grid):
    clf = RandomForestClassifier()
    grid_search = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)

    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    return grid_search, end_time - start_time

# Different values of N
N_values = [10, 100, 500, 1000]

for N in N_values:
    # Sample N images for cats and dogs
    cats_features = extract_features(cats_imgs[:N], fe_mod)
    dogs_features = extract_features(dogs_imgs[:N], fe_mod)

    # Create labels
    Y_cats = np.array(N * [1])
    Y_dogs = np.array(N * [0])

    # Stack features and labels
    X_cvd = np.vstack([cats_features, dogs_features])
    Y_cvd = np.hstack([Y_cats, Y_dogs])

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, test_size=0.2, random_state=42)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define the parameter grid for RandomForestClassifier (adjust as needed)
    param_grid = {
        'n_estimators': [10, 50, 100],
        'max_depth': [None, 10, 20],
    }

    # Train the classifier and measure computational time
    clf, training_time = train_classifier(X_train_scaled, y_train, param_grid)

    # Predict on the test set
    y_pred = clf.predict(X_test_scaled)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Report results
    print(f"\nResults for N = {N}:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Computational Time (seconds): {training_time:.2f}")

    # Print best hyperparameters from the grid search
    print("Best Hyperparameters:")
    print(clf.best_params_)



Results for N = 10:
Accuracy: 1.00
Computational Time (seconds): 4.41
Best Hyperparameters:
{'max_depth': 10, 'n_estimators': 10}

Results for N = 100:
Accuracy: 0.85
Computational Time (seconds): 5.27
Best Hyperparameters:
{'max_depth': 20, 'n_estimators': 50}

Results for N = 500:
Accuracy: 0.95
Computational Time (seconds): 26.51
Best Hyperparameters:
{'max_depth': 20, 'n_estimators': 100}

Results for N = 1000:
Accuracy: 0.95
Computational Time (seconds): 61.14
Best Hyperparameters:
{'max_depth': None, 'n_estimators': 50}
